In [1]:
from desc import set_device
set_device("gpu")

/home/fcastro/DESC/desc/__init__.py:99: UserWarning: No GPU found, falling back to CPU
  warnings.warn(colored("No GPU found, falling back to CPU", "yellow"))


In [2]:
import numpy as np
import os

from scipy.io import netcdf_file
import copy
import matplotlib
import matplotlib.pyplot as plt
import scipy
import sys
import functools
import pickle

import jax
import jax.numpy as jnp
from jax import jit, jacfwd

from netCDF4 import Dataset

from desc.backend import put, fori_loop, jnp, sign

from desc.basis import FourierZernikeBasis, DoubleFourierSeries, FourierSeries

from desc.equilibrium import EquilibriaFamily, Equilibrium
from desc.grid import ConcentricGrid, LinearGrid, Grid, QuadratureGrid
from desc.io import InputReader, load
#from desc.plotting import plot_1d, plot_2d, plot_3d, plot_section, plot_surfaces, plot_comparison

from desc.plotting import *

from desc.transform import Transform
from desc.vmec import VMECIO
from desc.derivatives import Derivative
from desc.geometry import FourierRZToroidalSurface
import desc.examples

from desc.backend import fori_loop, jit, jnp, odeint, sign
from desc.compute import rpz2xyz, rpz2xyz_vec, xyz2rpz, xyz2rpz_vec
from desc.coils import *

from desc.utils import flatten_list, Timer, copy_coeffs, errorif, setdefault, svd_inv_null, warnif, cross, dot

from desc.optimize import lsqtr, lsq_auglag

from desc.magnetic_fields import FourierCurrentPotentialField

import time

import numpy as np
from numpy import ndarray

from desc.fns_simp import (plot_figure,
                           plot_figure2,
                            plot_xy,
                           surf_int,
                           _compute_magnetic_field_from_Current)

#from find_sour import iso_coords_interp, bn_res, B_sour, K_sour, interp_grid, add_extra_coords, add_extra_periodic
from desc.objectives.find_sour import iso_coords_load, iso_coords_interp

ERROR:2025-08-18 15:37:32,755:jax._src.xla_bridge:444: Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/home/fcastro/.conda/envs/desc-env/lib/python3.12/site-packages/jax/_src/xla_bridge.py", line 442, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/home/fcastro/.conda/envs/desc-env/lib/python3.12/site-packages/jax_plugins/xla_cuda12/__init__.py", line 324, in initialize
    _check_cuda_versions(raise_on_first_error=True)
  File "/home/fcastro/.conda/envs/desc-env/lib/python3.12/site-packages/jax_plugins/xla_cuda12/__init__.py", line 281, in _check_cuda_versions
    local_device_count = cuda_versions.cuda_device_count()
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: jaxlib/cuda/versions_helpers.cc:113: operation cuInit(0) failed: Unknown CUDA error 303; cuGetErrorName failed. This probably means that JAX was unable to load the CUDA libraries.


SyntaxError: invalid syntax (_sources.py, line 332)

In [ ]:
# Plasma surfaces
#eqname_QA = "/scratch/gpfs/fcastro/LMStell/regcoil/rogerio/input.QA_final_output.h5" #vacuum equilibrium
eqname_QA = "input.QA_final_output.h5" #vacuum equilibrium
eq_QA = desc.io.load(eqname_QA)[4]

eq = eq_QA

name = './iso_coords/'
# Winding surfac./e
surf_winding = load(name + "surf.h5")

# Plot both surfaces
plot_comparison([surf_winding,eq],labels=["surf","eq"],theta=0,rho=np.array(1.0))

In [ ]:
tdata = iso_coords_load(name,surf_winding)

In [ ]:
# Evaluate Green's function for flat tori
sgrid = LinearGrid(M = 10, N = 15,
                   NFP = surf_winding.NFP,
                  )
sdata = surf_winding.compute(["theta","zeta", "e^theta_s","e^zeta_s"], grid = sgrid)
sdata = iso_coords_interp(tdata, sdata, sgrid,)

G = np.load(name + "G.npy")
K_sec = G * cross(sdata["n_rho"], sdata["e^u_s"])
K_sv = surf_winding.compute(["K"],grid = sgrid)["K"]

In [ ]:
sgrid2 = Grid(nodes = jnp.vstack((sgrid.nodes[:,0],
                                        sgrid.nodes[:,1],
                                        sgrid.nodes[:,2] + (2*np.pi/surf_winding.NFP)*1)).T
                    )
sgrid3 = Grid(nodes = jnp.vstack((sgrid.nodes[:,0],
                                        sgrid.nodes[:,1],
                                        sgrid.nodes[:,2] + (2*np.pi/surf_winding.NFP)*2)).T
                    )

sdata2 = surf_winding.compute(["theta","zeta", "e^theta_s","e^zeta_s"], grid = sgrid2)
sdata2 = iso_coords_interp(tdata, sdata2, sgrid2)

sdata3 = surf_winding.compute(["theta","zeta", "e^theta_s","e^zeta_s"], grid = sgrid3)
sdata3 = iso_coords_interp(tdata, sdata3, sgrid3)

In [ ]:
# Confirm that variables have the same shape
plot_figure2(sdata["u_iso"],sgrid,''r' $ u(\theta,\zeta)$ ')
plot_figure2(sdata["v_iso"],sgrid,''r' $ v(\theta,\zeta)$ ')
#plot_figure2(sdata["lambda_iso"],sgrid,''r' $ \lambda $ ')

#plot_figure2(sdata2["u_iso"],sgrid2,''r' $ u(\theta,\zeta)$ ')
#plot_figure2(sdata2["v_iso"],sgrid2,''r' $ v(\theta,\zeta)$ ')

#plot_figure2(sdata3["u_iso"],sgrid3,''r' $ u(\theta,\zeta)$ ')
#plot_figure2(sdata3["v_iso"],sgrid3,''r' $ v(\theta,\zeta)$ ')

In [ ]:
eps = 1e-2
dt = eps
dz = eps
d0 = eps/3

In [ ]:
grid_M = 10
grid_N = 10

In [ ]:
egrid = LinearGrid(M = grid_M, N = grid_N, NFP = eq.NFP)
edata = eq.compute(["n_rho","B"], grid = egrid)

#B_s = edata["B"]
B_sec = np.load('iso_coords/B_sec_'+str(grid_M)+'x'+str(grid_N)+'.npy')
#_compute_magnetic_field_from_Current( sgrid, K_sec, surf_winding, eq, egrid, basis = "rpz" )
B_s = _compute_magnetic_field_from_Current( sgrid, K_sv, surf_winding, eq, egrid, basis = "rpz" )
#B_s = _compute_magnetic_field_from_Current( sgrid, K_sec, surf_winding, eq, egrid, basis = "rpz" )
rhs = jnp.concatenate((B_s[:,0],B_s[:,1],B_s[:,2],
                      jnp.asarray([0])))
#- dot( edata["n_rho"],  B_sec )

In [ ]:
plot_figure2( dot(K_sec,K_sec) ** (1/2), sgrid,''r' $ | \mathbf{K_{sec}} |$ ')
plot_figure2( dot(B_s,B_s) ** (1/2), egrid,''r' $ | \mathbf{B_{s}} |$ ')
#plot_figure2( rhs, egrid, ''r' $ \mathbf{B_{sec} \cdot n}$ ' )

In [ ]:
N = 20 # Terms toa pproximate the infinite series

# Numer of dipoles
sMv = np.asarray([10,]) # Poloidal direction
sNv = np.asarray([10,]) # Toroidal direction

In [ ]:
p_M = sMv[0]
p_N = sNv[0]

#theta = jnp.linspace(2 * jnp.pi * (1 / (p_M * 2)) * 1/2,
#                         2 * jnp.pi * (1 - 1 / (p_M * 2) * 1/2),
#                         p_M * 2)

#zeta = jnp.linspace(2 * jnp.pi / surf_winding.NFP * (1 / (p_N * 2)) * 1/2,
#                        2 * jnp.pi / surf_winding.NFP * (1 - 1 / (p_N * 2) * 1/2),
#                        p_N * 2,)

#name = "iso_coords/"
#s_data = interp_grid(theta, zeta, surf_winding, name)

## Save the dictionary
#with open('s_data.pkl', 'wb') as f:
#    pickle.dump(s_data, f)

In [ ]:
grid_M2 = grid_M
grid_N2 = grid_N

egrid2 = LinearGrid(M = grid_M2, N = grid_N2, NFP = eq.NFP)
edata2 = eq.compute(["n_rho"], grid = egrid2)

B_sec2 = np.load('iso_coords/B_sec_'+str(grid_M2)+'x'+str(grid_N2)+'.npy')
#_compute_magnetic_field_from_Current( sgrid, K_sec, surf_winding, eq, egrid2, basis = "rpz" )
B_s2 = _compute_magnetic_field_from_Current( sgrid, K_sv, surf_winding, eq, egrid2, basis = "rpz" )
#B_s2 = edata2["B"]

In [ ]:
b_chi = [] # Store the integral of the error 
source_chi = [] # Store the sum of sinks and sources of current
Bn_chi = [] # Store the max Bn from the solution
max_I = [] # Store the max value of sources
min_I = [] # Store the min value of sources

In [ ]:
from desc.magnetic_fields._sinkssources import SinksSources

from desc.objectives import (
    ObjectiveFunction,
    SinksSourcesSurfaceQuadraticFlux,
)

In [ ]:
sinks_sources_basis = SinksSources(p_M=10,p_N=10, NFP = surf_winding.NFP)

In [ ]:
objective = ObjectiveFunction(
    (
        SinksSourcesSurfaceQuadraticFlux(
            sinks_sources_basis, # Field for sinks and sources,
            eq,
        surf_winding, # Winding surface
        tdata, # Dictionary with information about the isothermal coordinates
        N, # Nnumber of terms for the sum in the Jacobi-theta function
        d0, # Regularization radius for Guenther's function
        #normalize=True,
        #normalize_target=True,
        source_grid=sgrid, #Grid with the locations of the sources and sinks
        eval_grid=egrid,
        field_grid=sgrid,
        #sin_and_source_grid=None,
        vacuum=False,
        name="Sinks/Sources Quadratic flux",
        jac_chunk_size=None,)
    )
)

In [ ]:
from desc.optimize import Optimizer

optimizer = Optimizer("lsq-auglag")

In [ ]:
#(eq, field), _
result = optimizer.optimize(
    #(surf_winding,sinks_sources_basis),
    sinks_sources_basis,
    objective, 
    #constraints = None, 
    maxiter=1, 
    verbose=3
)